In [1]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf


In [2]:
conf = SparkConf()
# conf.set("spark.driver.memory", "4g")
# conf.set("spark.executor.memory", "4g")
conf.setMaster('local[8]')
conf.setAppName('Assignment_2')
sc = SparkContext.getOrCreate(conf)

In [127]:
import json
import math
import itertools
import random
import sys

In [4]:
reviews_json = sc.textFile('asnlib/publicdata/train_review.json').map(json.loads)

In [5]:
user_business_pairs = reviews_json.map(lambda x: (x.get('user_id'), x.get('business_id'))).distinct()

In [6]:
user_business_pairs.take(10)

[('OLR4DvqFxCKLOEHqfAxpqQ', 'zK7sltLeRRioqYwgLiWUIA'),
 ('0XMLbsJt-fvcQsOHt3_B_Q', 'eTXYID00jGxq1vZpntBUFw'),
 ('bQCHF5rn5lMI9c5kEwCaNA', 'na4Th5DrNauOv-c43QQFvA'),
 ('fiGqQ7pIGKyZ9G0RqWLMpg', 'IVieU4_z2Y9BsRANZYMNNg'),
 ('sDlRFzGHzfpFHaxvJ1JtTg', 'avkOIpslBUFZMQhELpl3JQ'),
 ('qRU7nYMJIV05lpKQq4jVjw', 'jobP3ywRd3QNZ_GCoPG2DQ'),
 ('d976WkRYJVqJyP3vH2Kayg', 'w6-qlTncaWlSxTJwK_HvfQ'),
 ('v5dxjxKLuINf5E6_kYckXg', 'MgSd4P3ATkiywbbraFygHg'),
 ('mLJ3zQwiEzAIBaXb-7iXBw', '3xykzfVY2PbdjKCRDLdzTQ'),
 ('PQ6-FZpUn2oFLyBGvdiG3w', 'bQ_wtZvMb__OhprY5bF9aQ')]

In [7]:
user_tokens_zip = user_business_pairs.map(lambda x: x[0]).distinct().sortBy(lambda x: x).zipWithIndex().map(lambda x: (x[0], x[1]+1))

In [8]:
user_tokens = user_tokens_zip.collectAsMap()

In [11]:
len(user_tokens)

26184

In [13]:
business_tokens_zip = user_business_pairs.map(lambda x: x[1]).distinct().sortBy(lambda x: x).zipWithIndex().map(lambda x: (x[0], x[1]+1))

In [14]:
business_tokens = business_tokens_zip.collectAsMap()

In [15]:
reversed_index_bus_dict = {v: k for k, v in business_tokens.items()}

In [19]:
len(business_tokens)

10253

In [151]:
def build_min_hash_func(a,b,p,m):
    def min_hash_func(x):
        return (((a*x + b)%p)%m)
#         return ((a*x + b)%m)
    return min_hash_func

def get_min_hash_functions(num_func, buckets):
    min_hash_func_list = []
    list_a = random.sample(range(1, sys.maxsize - 1), num_func)
    list_b = random.sample(range(0, sys.maxsize - 1), num_func)
    p = 233333333333
    for a,b in zip(list_a, list_b):
        min_hash_func_list.append(build_min_hash_func(a,b,p,buckets))
#         min_hash_func_list.append(lambda x: ((a*x + b)%p)%buckets)
#         print (a,b)
    return min_hash_func_list

In [152]:
min_hash_func_list = get_min_hash_functions(50, len(user_tokens)*2)

In [179]:
user_hashed_values = user_tokens_zip.map(lambda x: (user_tokens.get(x[0]), [min_hash(x[1]) for min_hash in min_hash_func_list]))

In [180]:
user_hashed_values.first()

(1,
 [37119,
  43659,
  26699,
  5115,
  30268,
  41878,
  50975,
  19196,
  25358,
  46619,
  9427,
  50246,
  4993,
  19972,
  33462,
  35685,
  3864,
  48151,
  22202,
  9992,
  22433,
  19324,
  15668,
  15283,
  38180,
  43722,
  4597,
  16589,
  11044,
  2874,
  32101,
  47578,
  8678,
  42714,
  24370,
  22531,
  26551,
  51040,
  50607,
  14128,
  43965,
  36589,
  33909,
  23397,
  3705,
  17629,
  19150,
  36024,
  44477,
  6093])

In [181]:
user_business_tokenized_map = user_business_pairs.map(lambda x: (user_tokens.get(x[0]), business_tokens.get(x[1]))).groupByKey().map(lambda x: (x[0], list(set(x[1]))))

In [182]:
user_business_tokenized_map.first()

(20664,
 [897,
  9223,
  8840,
  3337,
  4617,
  4363,
  9100,
  6667,
  777,
  6155,
  1809,
  275,
  1303,
  5275,
  3356,
  926,
  289,
  3365,
  807,
  1193,
  5804,
  3118,
  8373,
  1206,
  3128,
  3658,
  8397,
  7501,
  9935,
  8781,
  5967,
  2766,
  3158,
  7387,
  8803,
  8552,
  9577,
  234,
  7531,
  7275,
  6125,
  8174,
  9840,
  4721,
  1904,
  10225,
  6003,
  7032,
  3580,
  2045])

In [183]:
business_user_tokenized_map = user_business_pairs.map(lambda x: (business_tokens.get(x[1]), user_tokens.get(x[0]))).groupByKey().map(lambda x: (x[0], list(set(x[1]))))

In [184]:
business_user_tokenized_dict = business_user_tokenized_map.collectAsMap()

In [185]:
len(business_user_tokenized_dict[4396])

9

In [186]:
signature_matrix_rdd = user_business_tokenized_map.leftOuterJoin(user_hashed_values)

In [187]:
signature_matrix_rdd = signature_matrix_rdd.map(lambda x: x[1])

In [188]:
signature_matrix_rdd = signature_matrix_rdd.flatMap(lambda business_set: [(x, business_set[1]) for x in business_set[0]]) \

In [189]:
signature_matrix_rdd = signature_matrix_rdd.reduceByKey(lambda a,b: [min(x, y) for x, y in zip(a, b)]).coalesce(2)

In [190]:
signature_matrix_rdd.take(10)

[(9100,
  [552,
   1199,
   837,
   858,
   117,
   366,
   137,
   2005,
   53,
   78,
   3,
   1,
   125,
   635,
   1436,
   59,
   446,
   11,
   4,
   484,
   3058,
   478,
   917,
   105,
   145,
   72,
   238,
   29,
   176,
   526,
   197,
   829,
   484,
   158,
   820,
   495,
   857,
   133,
   266,
   691,
   707,
   1575,
   252,
   186,
   555,
   367,
   225,
   807,
   412,
   726]),
 (1904,
  [1866,
   1529,
   24,
   201,
   930,
   433,
   94,
   205,
   518,
   955,
   3023,
   680,
   129,
   19,
   809,
   443,
   295,
   86,
   301,
   358,
   85,
   452,
   203,
   2025,
   793,
   710,
   1108,
   280,
   43,
   99,
   344,
   236,
   136,
   541,
   8,
   1943,
   324,
   63,
   286,
   461,
   203,
   1044,
   1262,
   492,
   258,
   1537,
   331,
   163,
   958,
   339]),
 (6244,
  [33,
   82,
   232,
   349,
   60,
   361,
   312,
   205,
   127,
   501,
   260,
   216,
   82,
   479,
   11,
   52,
   164,
   3,
   78,
   11,
   28,
   80,
   135,
   86,
 

In [191]:
candidate_pairs = signature_matrix_rdd \
        .flatMap(lambda x: [(tuple([i, hash(tuple(x[1][i:i+2]))]), x[0]) for i in range(0, 50)]) \
        .groupByKey().map(lambda kv: list(kv[1])).filter(lambda val: len(val) > 1) \
        .flatMap(lambda bid_list: [pair for pair in itertools.combinations(bid_list, 2)])

In [193]:
candidate_pairs.take(10)

In [194]:
def computeJaccard(set1, set2):
    """
    compute Jaccard Similarity
    :param set1:
    :param set2:
    :return: a float number
    """
    return float(float(len(set(set1) & set(set2))) / float(len(set(set1) | set(set2))))

def verifySimilarity(candidate_pairs, index_data_dict,
                     reversed_index_dict, threshold):
    """
    iterate these candidate pairs,
            and compute the jaccard similarity from original data
    :param candidate_pairs: tuple(bidx1, bidx2)
    :param index_data_dict: dict(bidx: [uidx1, uidx2,...])
    :param reversed_index_dict: dict(bidx: bid_str)
    :param threshold: jaccard similarity threshold`
    :return: a list of dict which contain truly similar
            bidx pair and theirs similarity
    """
    result = list()
    temp_set = set()
    for pair in candidate_pairs:
        if pair not in temp_set:
            temp_set.add(pair)
            similarity = computeJaccard(index_data_dict.get(pair[0], set()),
                                        index_data_dict.get(pair[1], set()))
            if similarity >= threshold:
                result.append({"b1": reversed_index_dict[pair[0]],
                               "b2": reversed_index_dict[pair[1]],
                               "sim": similarity})
    return result


In [195]:
    result_list = verifySimilarity(candidate_pairs=set(candidate_pairs.collect()),
                                   index_data_dict=business_user_tokenized_dict,
                                   reversed_index_dict=reversed_index_bus_dict,
                                   threshold=0.05)


In [196]:
len(result_list)

13413

In [197]:
len(candidate_pairs_list)

178832

In [ ]:
candidate_pairs_list = candidate_pairs.collect()

In [ ]:
candidate_pairs.take(10)

In [ ]:
import math
value_list = [698,
   476,
   81,
   350,
   145,
   275,
   580,
   232,
   195,
   730,
   13,
   135,
   62,
   1029,
   955,
   252,
   786,
   97,
   282,
   46,
   0,
   154,
   51,
   313,
   596,
   249,
   231,
   319,
   143,
   464,
   60,
   24,
   156,
   39,
   804,
   1174,
   1067,
   26,
   265,1045,1028,602,301,113,1065,110,714,78,1948,76]
chunk_num = 10
chunk_lists = list()
size = int(math.ceil(len(value_list) / int(chunk_num)))

In [ ]:
size

In [ ]:
for index, start in enumerate(range(0, len(value_list), size)):
    chunk_lists.append((index, hash(tuple(value_list[start:start + size]))))

In [ ]:
chunk_lists

In [ ]:
[tuple([i//5, hash(tuple(value_list[i:i+5]))]) for i in range(0, 50, 5)]

In [ ]:
[list(range(0,50))[i:i + 5] for i in range(0, 50, 5)]